# PETs/TETs – Hyperledger Aries / PySyft – City (Relying Party) 🏙️

---
⚠️ <span style='background : yellow'>**Warning:**</span>

The `SyMPC` package is still in beta-verion and therefore very buggy. At the time of the development of this project, the function `.reconstruct()` (see Step ???) does not function with remote agents (i.e., one container per agent).

Thus, the notebooks under the directory `xx/` demonstrate the `.reconstruct()` function locally (within one docker container).

---

In [ ]:
%%javascript
document.title ='🏙️ City'

## PART 3: Connect with Manufacturers and Analyze Data

**What:** Obtain data from Manufacturers in a trust- and privacy-preserving manner

**Why:** Get manufacturers to share data anonymously to help the City analyze data

**How:** <br>
1. [Initiate City's AgentCommunicationManager (ACM)](#1)
2. [Connect with anonymous agents via a multi-use SSI invitation](#2)
3. [Request VCs to verify agents are certified manufacturers](#3)
4. [Join Duet Connections to obtain encrypted data](#4)

**Accompanying Agents and Notebooks:**
* Manufacturer1 🚗: `03_connect_with_city.ipynb`
* Manufacturer2 🚛: `03_connect_with_city.ipynb`
* Manufacturer3 🛵: `03_connect_with_city.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [ ]:
import os
import time

import syft as sy
from aries_cloudcontroller import AriesAgentController
from pprintpp import pprint
from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor
from termcolor import colored

import libs.helpers as helpers
from libs.agent_connection_manager import RelyingParty

#### 0.2 – Variables

In [ ]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1></a>

### 1 – Initiate City Agent
#### 1.1 – Init ACA-PY agent controller

In [ ]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [ ]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

#### 1.3 – Init ACM Relying Party

In [ ]:
# The CredentialHolder registers relevant webhook servers and event listeners
city_agent = RelyingParty(agent_controller)

---

<a id=2></a>

### 2 – Establish a connection with the Manufacturer agents

#### 2.1  – Create multi-use invitation
Send out a multi-use invitation to all manufacturer agents (i.e., copy & paste the same invitation to manufacturer1, manufacturer2, manufacturer3). This represents a scenario where the City agent invites any agent to connect with them, and authenticate as manufacturers. 

🕵️‍️ The advantage of a multi-use invitation is, that the City is unaware about who accessed the invitation link and is trying to get in contact with the City. The City agent only knows the names defined in `ACAPY_LABEL` (see respective `.env` files), and does not know which agent is which manufacturer.

**Note:** Please establish a connection with all three manufacturer agents.

In [ ]:
# Setup for connection with Manufacturer agents
alias = "Connection 1"
auto_accept = True # Accept response of Manufacturers agent right away
auto_ping = True
public = False # Do not use public DID
multi_use = True # Invitation can be used by multiple inviteees

invitation = city_agent.create_connection_invitation(alias=alias, auto_accept=auto_accept, public=public, multi_use=multi_use, auto_ping=auto_ping)

<div style="font-size: 25px"><center><b>Break Point 1</b></center></div>
<div style="font-size: 50px"><center>🏙️ ➡️ 🚗 / 🚛 / 🛵</center></div><br>
<center><b>Please open all manufacturer agents 🚗 / 🚛 / 🛵. <br> For each of the manufacturer agents, open the 03_connect_with_city.ipynb notebook and execute all steps until Break Point 2/3/4. <br> Use the same invitation from the City Agent 🏙️ in Step 2.1 for all Manufacturers.</b></center>


#### 2.2 – Display all active connections

In [ ]:
# Display all active connections
for conn in city_agent.get_active_connections():
    conn.display()

#### 2.3 – Fetch all connection_ids of the active connections with the respective manufacturer agents
Fetch the connection_ids with the name provided in the active connections (see print from previous cell).


In [ ]:
# Get connection_id with Manufacturer1
connection_id_a1 = city_agent.get_connection_id("AnonymousAgent1")[-1] # We assume that there is only one connection with Manufacturer1

# Get connection_id with Manufacturer2
connection_id_a2 = city_agent.get_connection_id("AnonymousAgent2")[-1] # We assume that there is only one connection with Manufacturer2

# Get connection_id with Manufacturer3
connection_id_a3 = city_agent.get_connection_id("AnonymousAgent3")[-1] # We assume that there is only one connection with Manufacturer3

<a id=3></a>
### 3 – Send Proof Request

#### 3.1 Define VC Presentation Request Object

The below cell defines a generic presentation request object, that can be sent across specific connections.
In this case, the City agent requests the Manufacturers to certify the requested attribute `isManufacturer` from the manufacturer schema.


In [ ]:
# Define which scheme we want a proof from
identifiers = helpers.get_identifiers()
schema_manufacturer_id = identifiers["manufacturer_schema_identifiers"]["schema_id"]

# Define the attributes we want from the scheme
req_attrs = [{"name": "isManufacturer", "restrictions": [{"schema_id": schema_manufacturer_id}]}]

# Define proof_request
# The proof_request irgnores predicates (e.g., range proofs) revokation
manufacturer_proof_request = {
    "name": "isManufacturer Proof Request",
    "version": "1.0",
    "requested_attributes": {f"0_{req_attr['name']}_uuid": req_attr for req_attr in req_attrs },
    "requested_predicates": {}, #{f"0_{req_pred['name']}_GE_uuid": req_pred for req_pred in req_preds },
    "non_revoked":  {"to": int(time.time())}
}

print(colored("Manufacturer Proof Request:", attrs=["bold"]))
pprint(manufacturer_proof_request)

#### 3.2 – Send Proof Request
The proof request asks the agent at the other end of `connection_id` to prove attributes defined within `manufacturer_proof_request` using the manufacturer schema.

The resulting presentation request is encoded in base64 and packed into a DIDComm Message. The `@type` attribute in the presentation request defines the protocol present-proof and the message request-presentation.

Overall, the proof-presentation procedure has six steps. **R** represents the Relying Party (here the City agent), and **H** the Holder (i.e., the manufacturers).

| Step | Status | Agent | Description | Function |
| --- | --- | --- | --- | --- |
| 1 | `request_sent` |  R | R requests a proof defined in `manufacturer_proof_request` | `send_proof_request()` |
| 2 | `request_received` | H | H receives a proof request from R | - |
| 3 | `presentation-sent` | H | H sends proof presentation to R | `send_proof_presentation()` |
| 4 | `presentation-received` | R  | R receives presentation from H | - |
| 5 | `verified` | R | R verifies presentation received from H | `verify_proof_presentation()` |
| 6 | `presentation_acked` | H | H knows, that R verified the presentation | - |


In [ ]:
# Send proof request to Manufacturer1 and get presentation_exchange_id for connection with manufacturer 1
presentation_exchange_id_a1 = city_agent.send_proof_request(
    connection_id=connection_id_a1, 
    proof_request=manufacturer_proof_request, 
    comment="Please prove that you are an agent who is a certified manufacturer"
)

# Send proof request to Manufacturer2 and get presentation_exchange_id for connection with manufacturer 2
presentation_exchange_id_a2 = city_agent.send_proof_request(
    connection_id=connection_id_a2, 
    proof_request=manufacturer_proof_request, 
    comment="Please prove that you are an agent who is a certified manufacturer"
)

# Send proof request to Manufacturer3 and get presentation_exchange_id for connection with manufacturer 3
presentation_exchange_id_a3 = city_agent.send_proof_request(
    connection_id=connection_id_a3, 
    proof_request=manufacturer_proof_request, 
    comment="Please prove that you are an agent who is a certified manufacturer"
)

<div style="font-size: 25px"><center><b>Break Point 5</b></center></div>
<div style="font-size: 50px"><center>🏙️ ➡️ 🚗 / 🚛 / 🛵</center></div><br>
<center><b>Please go to all manufacturer agents 🚗 / 🚛 / 🛵. <br> For each of the manufacturer agents, continue with executing Step 3</b></center>

#### 3.3 – Verify Proof Presentations

When proof presentations are received by the manufacturers (see output under Step 3.2), you can verify whether the proof presentations are valid. 
A presentation object contains three classes of attributes. 
* Revealed Attributes: Attributes that were signed by an issuer and have been revealed in the presentation process
* Self Attested Attributes: Attributes that the prover has self attested to in the presentation object.
* Predicate proofs: Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

Execute the following cell. The `verify_proof_presentation()` function verifies the proof presentation, and parses out relevant information. The returning value states whether the presentation proof is valid or not. In the use case at hand, because only one variable is verified, the boolean denotes if the agent is a manufacturer or not.

In [ ]:
a1_is_manufacturer = city_agent.verify_proof_presentation(presentation_exchange_id_a1)
a2_is_manufacturer = city_agent.verify_proof_presentation(presentation_exchange_id_a2)
a3_is_manufacturer = city_agent.verify_proof_presentation(presentation_exchange_id_a3)

print("AnonymousAgent1 is manufacturer:", a1_is_manufacturer)
print("AnonymousAgent2 is manufacturer:", a2_is_manufacturer)
print("AnonymousAgent3 is manufacturer:", a3_is_manufacturer)

<a id=4></a>
### 4 – Do Data Science
Now that the City agent verified that all agents are indeed manufactureres, join the individual Duet Connections that the Manufacturers already sent to the City Agent.

#### 4.1 – Establish a Duet connections
Duet is a package that allows you to exchange encrypted data and run privacy-preserving arithmetic operations on them (e.g., through homomorphic encryption or secure multiparty computation). The ACM package is configured to allow the exchange of duet tokens. Only one duet connection can be established per aries connection.

##### Duet Connection with AnonymousAgent1

In [ ]:
# Set up connection_id used for for duet token exchange
city_agent._update_connection(connection_id=connection_id_a1, is_duet_connection=True, reset_duet=True)

# Join duet established by external agent 
duet_a1 = sy.join_duet(credential_exchanger=city_agent)

# Check that you can access the duet data store
duet_a1.store.pandas

##### Duet Connection with AnonymousAgent2

In [ ]:
# Set up connection_id used for for duet token exchange and join duet
city_agent._update_connection(connection_id=connection_id_a2, is_duet_connection=True, reset_duet=True)
duet_a2 = sy.join_duet(credential_exchanger=city_agent)

# Check that you can access the duet data store
duet_a2.store.pandas

##### Duet Connection with AnonymousAgent3

In [ ]:
# Set up connection_id used for for duet token exchange and join duet
city_agent._update_connection(connection_id=connection_id_a3, is_duet_connection=True, reset_duet=True)
duet_a3 = sy.join_duet(credential_exchanger=city_agent)

# Check that you can access the duet data store
duet_a3.store.pandas

#### 4.2 – Setup Secure Multiparty Computation (SyMPC) Session with all duet connections
Create a duet session that is able to access the data from all three established duet connections.

Then, initiate the duet session to enable Secure Multiparty Computation. The `Session` is used to send some config information only once between the parties. This information can be:
* the ring size in which we do the computation
* Reference to the parties involved
* the precision and base
* ...

The `MPCTensor` is the tensor that holds reference to the shares owned by the different parties. Specifically, it is an orchestrator that can do computations on data that it does not see.

In [ ]:
session = Session(parties=[duet_a1, duet_a2, duet_a3])
SessionManager.setup_mpc(session)

session.get_protocol()

<div style="font-size: 25px"><center><b>Break Point 9</b></center></div>
<div style="font-size: 50px"><center>🏙️ ➡️ 🚗 / 🚛 / 🛵</center></div><br>
<center><b>Please go to all manufacturer agents 🚗 / 🚛 / 🛵. <br> For each of the manufacturer agents, continue with executing Step 4.2</b></center>

#### 4.3 – Access encrypted data from AnonymousAgent1 🔒🚗, AnonymousAgent2 🔒🚛, and AnonymousAgent3 🔒🛵
Access data from the encrypted duet stores by ID. Check which data the stores have to offer, and enter the names of the data entries to retrieve them

In [ ]:
duet_a1.store.pandas

In [ ]:
duet_a2.store.pandas

In [ ]:
duet_a3.store.pandas

In [ ]:
# Retrieve encrypted data
x_secret = duet_a1.store["hourly-co2-per-zip_2021-08-19"] # describe local data to test sum, substract, and multiply
y_secret = duet_a2.store["hourly-co2-per-zip_2021-08-19"]
z_secret = duet_a3.store["hourly-co2-per-zip_2021-08-19"]

In [ ]:
# Test the output of one of them
print(x_secret)

#### 4.4 – Share secrets with Agents in session

In [ ]:
# Convert encrypted data into MPCTensors and share the secrets
x = MPCTensor(secret=x_secret, shape=(1,), session=session) # @todo: adjust shape!
y = MPCTensor(secret=y_secret, shape=(1,), session=session)
z = MPCTensor(secret=z_secret, shape=(1,), session=session)

In [ ]:
# Test the output of one of them
print(z)

#### 4.5 – Reconstruct data shares 🔐(🚗 + 🚛 + 🛵)
Let's first do some basic operations. Because these operations are performed via SMPC, the raw data is not leaving the data owners' servers!

Unfortunately, the following lines of codes always receive an error message.
[See Github issue](https://github.com/OpenMined/SyMPC/issues/282). If you want to see how the `.reconstruct()` function works, go to `xx/` and run the notebook.

In [ ]:
# Do basic arithmetic operations on the MPCTensors
print("X + Y + Z = ",(x + y + z).reconstruct())

---

### 5 – Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()

---

## 🔥🔥🔥 You can close this notebook now 🔥🔥🔥